In [11]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
#
import xgboost
#
import importlib
import utilities.train_test as train_test
import utilities.comparison as comparison

In [ ]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

min_date = pd.to_datetime(df.index.max()) - pd.DateOffset(months=12*15)
min_datestr = min_date.strftime('%Y-%m-%d')

#df = df.loc[df.index >= min_datestr]

In [12]:
importlib.reload(train_test)

df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

file=Path("../../../data/df_tabular_multi.csv")

if file.exists():
    df_tabular=pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular = train_test.get_dataframe_tabular_multi(df)
    df_tabular.to_csv(file)

## Direct forecasting

In [30]:
# True values
y_true = df.mean(axis=1)

## 1 Month

In [ ]:
importlib.reload(train_test)

X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60)

model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

In [ ]:
importlib.reload(comparison)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m = train_test.get_train_test(df_tabular, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

In [ ]:
# Predictions
y_train_pred_1m = model.predict(X_train_1m)
y_test_pred_1m = model.predict(X_test_1m)

#### Train data

In [ ]:
importlib.reload(comparison)
y_train_mean_pred_1m, y_test_mean_pred_1m = comparison.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m, len(df.columns))

### 1 month Actual vs Prediction

In [ ]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_1m, y_test_mean_pred_1m)

#### Overview table

In [ ]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = comparison.get_df_to_evaluate(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

In [ ]:
importlib.reload(comparison)
df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = comparison.get_portfolio_performance(df_to_evaluate_1m,
                                                                                                    "xgbooster_weights_1m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)

allocation, leftover = comparison.create_discrete_allocation(df_pred_1m, raw_weights_1m)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m

## 6 Months


In [ ]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60, target_key='m_return_target(t+6)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

In [ ]:
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m = train_test.get_train_test(df_tabular, months=6,
                                                                         target_key='m_return_target(t+6)')

# Predictions
y_train_pred_6m = model.predict(X_train_6m)
y_test_pred_6m = model.predict(X_test_6m)

#### Train data

In [ ]:
importlib.reload(comparison)
y_train_mean_pred_6m, y_test_mean_pred_6m = comparison.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df.columns))

### 6 Months Actual vs Prediction

In [1]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_6m, y_test_mean_pred_6m)

NameError: name 'importlib' is not defined

#### Overview

In [ ]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_test_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = comparison.get_df_to_evaluate(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

In [ ]:
importlib.reload(comparison)
df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = comparison.get_portfolio_performance(df_to_evaluate_6m,
                                                                                                    "xgbooster_weights_6m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)
comparison.create_discrete_allocation(df_pred_6m, raw_weights_6m)

# Create overview
df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m

## 12 Months

In [ ]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60, target_key='m_return_target(t+12)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

In [ ]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m = train_test.get_train_test(df_tabular, months=12,
                                                                             target_key='m_return_target(t+12)')

In [ ]:
# Predictions
y_train_pred_12m = model.predict(X_train_12m)
y_test_pred_12m = model.predict(X_test_12m)

#### Train-data

In [ ]:
importlib.reload(comparison)
y_train_mean_pred_12m, y_test_mean_pred_12m = comparison.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df.columns))

### 12 Months Actual vs Prediction

In [ ]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_12m, y_test_mean_pred_12m)

#### Overview table

In [ ]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = comparison.get_df_to_evaluate(df, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

In [ ]:
importlib.reload(comparison)
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = comparison.get_portfolio_performance(df_to_evaluate_12m,
                                                                                                          "xgbooster_weights_12m.csv",
                                                                                                          min_avg_return=-0.5,
                                                                                                          months=12)
comparison.create_discrete_allocation(df_pred_12m, raw_weights_12m)

# Create overview
df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
# 
df_view_12m['return_last_period(12m)'] = round(df.tail(12).prod() - 1, 2)
df_view_12m